In [1]:
import torch
print(torch.__version__)  # 打印 PyTorch 版本
print(torch.cuda.is_available())  # 检查 CUDA 是否可用
print(torch.version.cuda)  # 打印 PyTorch 使用的 CUDA 版本

2.6.0+cu118
True
11.8


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}", torch.cuda.is_available())

Using device: cuda True


In [7]:
import torch.nn as nn
import torch.optim as optim
import scipy.io
import numpy as np
import time
import matplotlib.pyplot as plt

In [9]:
class PINN(nn.Module):
	def __init__(self, weightF, weightU, weightS, weightB):
		super(PINN, self).__init__()
		self.fc1 = nn.Linear(3,200)
		self.fc2 = nn.Linear(200,200)
		self.fc3 = nn.Linear(200,200)
		self.fc4 = nn.Linear(200,200)
		self.fc5 = nn.Linear(200,1)
		self.c = nn.Parameter(torch.randn(30,30)*0.1+1.5)
		
		# Gain for tanh
		gain = nn.init.calculate_gain('tanh')
		
		nn.init.kaiming_normal_(self.fc1.weight, mode='fan_in', nonlinearity='tanh', a=gain)
		nn.init.kaiming_normal_(self.fc2.weight, mode='fan_in', nonlinearity='tanh', a=gain)
		nn.init.kaiming_normal_(self.fc3.weight, mode='fan_in', nonlinearity='tanh', a=gain)
		nn.init.kaiming_normal_(self.fc4.weight, mode='fan_in', nonlinearity='tanh', a=gain)
		nn.init.kaiming_normal_(self.fc5.weight, mode='fan_in', nonlinearity='tanh', a=gain)
		
		self.weightF = weightF
		self.weightU = weightU
		self.weightS = weightS
		self.weightB = weightB
		
	def forward(self, x,y,t):
		inputs = torch.cat([x,y,t], dim=1)
		u1 = torch.tanh(self.fc1(inputs))
		u2 = torch.tanh(self.fc2(u1))
		u3 = torch.tanh(self.fc3(u2))
		u4 = torch.tanh(self.fc4(u3))
		u = self.fc5(u4)
		return u
		
	def mse_loss(self, output, target):
		return torch.mean((output - target)**2)  

	def calculate_derivatives(self,x,y,t):
		u = self.forward(x,y,t)
		
		u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
		u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
		u_y = torch.autograd.grad(u.sum(), y, create_graph=True)[0]
		
		u_tt = torch.autograd.grad(u_t.sum(), t, create_graph=True)[0]
		u_xx = torch.autograd.grad(u_x.sum(), x, create_graph=True)[0]
		u_yy = torch.autograd.grad(u_y.sum(), y, create_graph=True)[0]
		
		laplacian_u = u_xx+u_yy
		return u_tt, laplacian_u
		
	def prepare_c_for_operations(self):
		num_time_steps=  100
		c_expanded = self.c.unsqueeze(2)
		c_repeated = c_expanded.repeat(1,1, num_time_steps)
		c_final = c_repeated.flatten()
		c_final=  c_final[:,None]
		c_squared = c_final**2
		return c_squared
		
	def loss_function(self, x,y,t,u_known,c_known):
		u_pred = self.forward(x,y,t)
		u_tt, laplacian_u = self.calculate_derivatives(x,y,t)
		c_squared = self.prepare_c_for_operations()
		weighted_laplacian = laplacian_u*c_squared
		lossF = self.mse_loss(u_tt-weighted_laplacian, torch.zeros_like(u_tt))
		lossU = self.mse_loss(u_pred,u_known)
		lossS = torch.relu(-self.c).sum()
		lossB = self.mse_loss(self.c[:,0],c_known[:,0]) + self.mse_loss(self.c[:,-1],c_known[:,-1]) + self.mse_loss(self.c[0,:], c_known[0,:])+self.mse_loss(self.c[-1,:],c_known[-1,:])
		total_loss = self.weightF*lossF+self.weightU*lossU+self.weightS*lossS+self.weightB*lossB
		return total_loss, lossF, lossU, lossS, lossB

In [11]:
def load_data():
	data = scipy.io.loadmat('Waves_NoAtten.mat')
	wavefield=  torch.tensor(data['W30_2'], dtype = torch.float32)
	wavefield = wavefield[:,:,:100]
	c_in = torch.tensor(data['c'], dtype=torch.float32)
	dt = 0.01
	dx=dy=0.1
	t = dt*torch.linspace(0,99,100, requires_grad=True)
	x = dx*torch.linspace(0,29,30, requires_grad=True)
	y = dy*torch.linspace(0,29,30, requires_grad=True)
	x,y,t = torch.meshgrid(x,y,t,indexing='ij')
	x = x.flatten().unsqueeze(1)
	y = y.flatten().unsqueeze(1)
	t = t.flatten().unsqueeze(1)
	u_known = wavefield.flatten().unsqueeze(1)
	c_known = c_in.to(device)
	return x,y,t,u_known,c_known

In [13]:
class CustomLRScheduler(torch.optim.lr_scheduler._LRScheduler):
	def __init__(self, optimizer, initial_lr, decay_rate, last_epoch=-1, verbose=False):
		self.initial_lr=  initial_lr
		self.decay_rate = decay_rate
		super(CustomLRScheduler, self).__init__(optimizer, last_epoch, verbose)
	def get_lr(self):
		return [self.initial_lr/(1+self.decay_rate*self.last_epoch) for group in self.optimizer.param_groups]

In [ ]:
x,y,t,u_known,c_known = load_data()
x = x.to(device)
y = y.to(device)
t = t.to(device)

u_known_normalized = 2*(u_known-u_known.min())/(u_known.max()-u_known.min()) - 1
print(u_known_normalized.max, u_known_normalized.min)
u_known = u_known_normalized
u_known = u_known.to(device)
c_known = c_known.to(device)

model = PINN(weightF=1, weightU=10, weightS=1, weightB=10).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = CustomLRScheduler(optimizer, initial_lr=1e-3, decay_rate=2e-4)

epochs = 60000
losses_total=[]
losses_F=[]
losses_U=[]
losses_S=[]
losses_B=[]
diffs_c=[]
start_time = time.time()
for epoch in range(epochs):
	optimizer.zero_grad()
	total_loss, lossF, lossU, lossS ,lossB = model.loss_function(x,y,t,u_known,c_known)
	total_loss.backward(retain_graph=True)
	optimizer.step()
	scheduler.step()
	current_lr = scheduler.get_last_lr()[0]
	
	losses_total.append(total_loss.item())
	losses_F.append(lossF.item())
	losses_U.append(lossU.item())
	losses_S.append(lossS.item())
	losses_B.append(lossB.item())
	diff_c = model.mse_loss(c_known[1:-1,1:-1], model.c[1:-1,1:-1]).detach().cpu().numpy().item()
	diffs_c.append(diff_c)
	if epoch%5==0:
		stop_time = time.time()
		e_time = stop_time - start_time
		print(f"Epoch {epoch}: Total Loss = {total_loss.item():.6g}, lossF = {lossF.item():.6g}, lossU = {lossU.item():.6g}, lossS={lossS.item():.6g}, lossB = {lossB.item():.6g}, elapsed time = {e_time:.6g}, diff_c = {diff_c:.6g}, lr = {current_lr:.6g}")
		start_time = time.time()
		
torch.save(model.state_dict(), 'pinn_model_scheduler.pth')
torch.save(model.c, 'learned_c_scheduler.pth')

loss_arr_total = np.array(losses_total)
loss_arr_U = np.array(losses_U)
loss_arr_F = np.array(losses_F)
loss_arr_S = np.array(losses_S)
loss_arr_B = np.array(losses_B)
diffs_arr_c = np.array(diffs_c)

try:
	np.savez('losses_archive_scheduler.npz', losses_Total = loss_arr_total, losses_U = loss_arr_U, losses_F = loss_arr_F, losses_S = loss_arr_S, losses_b = loss_arr_B, diffs_c = diffs_arr_c)
	print('Losses saved successfully')
except Exception as e:
	print(f'An error occurred:{e}')


<built-in method max of Tensor object at 0x000002B03683F110> <built-in method min of Tensor object at 0x000002B03683F110>


C:\Users\13391\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0: Total Loss = 57.3629, lossF = 34.3007, lossU = 2.02836, lossS=0, lossB = 0.277859, elapsed time = 2.55777, diff_c = 0.0507801, lr = 0.0009998
Epoch 5: Total Loss = 14.0562, lossF = 8.52854, lossU = 0.282099, lossS=0, lossB = 0.270665, elapsed time = 22.4597, diff_c = 0.0515029, lr = 0.000998801
Epoch 10: Total Loss = 8.32142, lossF = 3.55936, lossU = 0.213063, lossS=0, lossB = 0.263143, elapsed time = 20.368, diff_c = 0.0520422, lr = 0.000997805
Epoch 15: Total Loss = 6.08029, lossF = 1.77742, lossU = 0.174729, lossS=0, lossB = 0.255558, elapsed time = 22.8745, diff_c = 0.0524526, lr = 0.00099681
Epoch 20: Total Loss = 5.53664, lossF = 1.43312, lossU = 0.162292, lossS=0, lossB = 0.24806, elapsed time = 23.134, diff_c = 0.0527741, lr = 0.000995818
Epoch 25: Total Loss = 4.62352, lossF = 0.703543, lossU = 0.151296, lossS=0, lossB = 0.240701, elapsed time = 24.1996, diff_c = 0.0530285, lr = 0.000994827
Epoch 30: Total Loss = 4.24493, lossF = 0.401195, lossU = 0.150873, lossS=0, l

In [59]:
# 打印初始内存使用情况
print(f"Initial memory allocated: {torch.cuda.memory_allocated(device)}")
print(f"Initial memory reserved: {torch.cuda.memory_reserved(device)}")

Initial memory allocated: 3989504000
Initial memory reserved: 15445524480


In [5]:
import torch

# 检查 GPU 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 检查 GPU 总内存
if device.type == "cuda":
    print(f"GPU total memory: {torch.cuda.get_device_properties(device).total_memory / 1024**3:.2f} GB")

# 清空 GPU 缓存
torch.cuda.empty_cache()

# 定义模型
model = torch.nn.Linear(10, 10).to(device)

# 估算模型内存
def estimate_model_memory(model):
    total_memory = 0
    for param in model.parameters():
        total_memory += param.numel() * param.element_size()
    return total_memory

print(f"Model memory usage: {estimate_model_memory(model)} bytes")

Using device: cuda
GPU total memory: 8.00 GB
Model memory usage: 440 bytes


In [3]:
print(torch.cuda.is_available())  # 应该返回 True
print(torch.cuda.get_device_name(0))  # 应该返回你的 GPU 名称

True
NVIDIA GeForce RTX 4060 Laptop GPU
